**Learning objectives**

* Implement the Deutsch-Jozsa algorithm.
* Compare the algorithmic performance of Deutsch-Jozsa to deterministic and classically random strategies.

In [1]:
import numpy as np
import pennylane as qml

![circuit](./images/A.6.1.png)

**Codercise A.6.1.** Implement the circuit above for a set of solutions `combos`, and return probabilities. As before, you are given `multisol_oracle_matrix(combos)`, which returns the associated oracle in matrix form.

Hitting submit will plot the probability of observing $0$ as a function of the size of the solution set; as for a single solution, this does not depend on the secret combinations themselves. What pattern do you observe?

In [2]:
def multisol_oracle_matrix(combos):
    """Return the oracle matrix for a set of solutions.

    Args:
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    indices = [np.ravel_multi_index(combo, [2]*len(combo)) for combo in combos]
    my_array = np.identity(2**len(combos[0])) # Create the identity matrix
    for i in indices:
        my_array[i,i] = -1
    return my_array


In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def multisol_hoh_circuit(combos):
    """A circuit which applies Hadamard, multi-solution oracle, then Hadamard.

    Args:
        combos (list[list[int]]): A list of secret bit strings.

    Returns:
        array[float]: Probabilities for observing different outcomes.
    """
    qml.broadcast(qml.Hadamard,pattern="single",wires=[i for i in range(n_bits)])

    oracle = multisol_oracle_matrix(combos)
    qml.QubitUnitary(oracle,wires=[i for i in range(n_bits)])

    qml.broadcast(qml.Hadamard, pattern="single", wires=[i for i in range(n_bits)])

    return qml.probs(wires=range(n_bits))

![circuit](./images/A.6.1.2.png)

The probability of observing $0$ appears to be $1$ when there are no solutions, and decreases until it hits zero at the halfway point. It then climbs symmetrically back up again to hit $1$ when all the combinations are solutions. The probability to observe $0$ turns out to be $|A_0|^2$, where
![circuit](./images/A.6.1.3.png)
for a set od solutions $S$ and non-solutions $T$. If half the combinations are solutions, $|S| = |T| = 2^{n-1}$ then $|A_0|^2 = 0$. We call such a function *balanced*. If all or none are solutions, $|S| = 0$ or $|S| = 2^n$ then $$|A_0|^2$ = 1$, and we call the function $constant$, since it always output the same bit.

**Codercise A.6.2.** Implement the Deutsch-Jozsa algorithm. Given a constant or balanced function based on the `promise_var` flag, implement the circuit from above to determine which it is. The function `multisol_hoh_circuit` from the previous exercise is available for you to use.

In [ ]:
def deutsch_jozsa(promise_var):
    """Implement the Deutsch-Jozsa algorithm and guess the promise variable.

    Args:
        promise_var (int): Indicates whether the function is balanced (0) or constant (1).

    Returns:
        int: A guess at the promise variable.
    """
    if promise_var == 0:
        how_many = 2**(n_bits - 1)
    else:
        how_many = np.random.choice([0, 2**n_bits]) # Choose all or nothing randomly
    combos = multisol_combo(n_bits, how_many) # Generate random combinations

    probs = multisol_hoh_circuit(combos)
    if np.isclose(probs[0], 1):
        return 1
    else:
        return 0

